In [7]:
%pylab inline
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

from IPython.core.magic import register_cell_magic
import pandas as pd
import numpy as np
import xgboost

Populating the interactive namespace from numpy and matplotlib


In [8]:
bechdelDataset = pd.read_csv('mergeWithRevenue.csv', encoding='utf-8') 
bechdelDataset.head(10)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,index,id,...,producer,writer,box_office,parental_rating,production_company,adjusted_box_office,Multigenre,director_name%,writer%,producer%
0,0,0,0,0,0,0,0,0,0,5433,...,Julien Pappííëíííí«ííí...,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
1,1,1,1,1,1,1,1,1,1,5444,...,Louis Lumiííëíííí«ííí...,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
2,2,2,2,2,2,2,2,2,2,6200,...,Thomas A. Edison,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
3,3,3,3,3,3,3,3,3,3,6199,...,NaN,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
4,4,4,4,4,4,4,4,4,4,5406,...,Georges Mííëíííí«íííí...,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.0,0.058824
5,5,5,5,5,5,5,5,5,5,4982,...,Georges Mííëíííí«íííí...,Georges Mííëíííí«íííí...,NaN,NaN,NaN,0.0,Y,0.032258,0.0,0.058824
6,6,6,6,6,6,6,6,6,6,5445,...,Lííëíííí«ííííëíí...,Alice Guy,NaN,NaN,NaN,0.0,Y,0.000000,0.0,0.000000
7,7,7,7,7,7,7,7,7,7,5411,...,NaN,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.0,0.000000
8,8,8,8,8,8,8,8,8,8,5410,...,Georges Mííëíííí«íííí...,Georges Mííëíííí«íííí...,NaN,NaN,Kadokawa Pictures,0.0,Y,0.032258,0.0,0.058824
9,9,9,9,9,9,9,9,9,9,4978,...,Georges Mííëíííí«íííí...,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.0,0.058824


In [10]:
#use this to construct pass percentages for each director/writer/producer 
#over their work in the dataset
from collections import defaultdict

def returnValue(x, counter):
    if (pd.notnull(x)):
       return counter[x]
    else:
       return 0.0

def constructRecord(dataset, role):
    columnName = role
    passValues = dataset[dataset.binary == 'PASS'][columnName].value_counts().to_dict()
    totalValues = dataset[columnName].value_counts().to_dict()
    passDict = defaultdict(float)
    for value in passValues:
        passDict[value] = passValues[value] / totalValues[value]
           
    dataset[columnName+'%'] = dataset[columnName].apply(lambda x: returnValue(x, passDict))  
    return dataset


In [12]:
bechdelDataset = constructRecord(bechdelDataset, 'director_name')
bechdelDataset = constructRecord(bechdelDataset, 'writer')
bechdelDataset = constructRecord(bechdelDataset, 'producer')

In [13]:
bechdelDataset.head(40)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,index,id,...,producer,writer,box_office,parental_rating,production_company,adjusted_box_office,Multigenre,director_name%,writer%,producer%
0,0,0,0,0,0,0,0,0,0,5433,...,Julien Pappííëíííí«ííí...,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
1,1,1,1,1,1,1,1,1,1,5444,...,Louis Lumiííëíííí«ííí...,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
2,2,2,2,2,2,2,2,2,2,6200,...,Thomas A. Edison,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
3,3,3,3,3,3,3,3,3,3,6199,...,NaN,NaN,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
4,4,4,4,4,4,4,4,4,4,5406,...,Georges Mííëíííí«íííí...,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.000000,0.058824
5,5,5,5,5,5,5,5,5,5,4982,...,Georges Mííëíííí«íííí...,Georges Mííëíííí«íííí...,NaN,NaN,NaN,0.0,Y,0.032258,0.000000,0.058824
6,6,6,6,6,6,6,6,6,6,5445,...,Lííëíííí«ííííëíí...,Alice Guy,NaN,NaN,NaN,0.0,Y,0.000000,0.000000,0.000000
7,7,7,7,7,7,7,7,7,7,5411,...,NaN,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.000000,0.000000
8,8,8,8,8,8,8,8,8,8,5410,...,Georges Mííëíííí«íííí...,Georges Mííëíííí«íííí...,NaN,NaN,Kadokawa Pictures,0.0,Y,0.032258,0.000000,0.058824
9,9,9,9,9,9,9,9,9,9,4978,...,Georges Mííëíííí«íííí...,NaN,NaN,NaN,NaN,0.0,Y,0.032258,0.000000,0.058824


In [15]:
bechdelDataset.to_csv('mergeWithRevenue.csv', encoding='utf-8')

In [23]:
names = bechdelDataset['producer'].unique()
producerFirstNames = []
for name in names:
    if (pd.notnull(name)):
        first = name.split(' ')
        producerFirstNames.append(first[0])
        
print(len(set(producerFirstNames)))

1792


In [25]:
names = bechdelDataset['writer'].unique()
writerFirstNames = []
for name in names:
    if (pd.notnull(name)):
        first = name.split(' ')
        writerFirstNames.append(first[0])
        
print(len(set(writerFirstNames)))

1935


In [26]:
names = bechdelDataset['director_name'].unique()
producerFirstNames = []
for name in names:
    if (pd.notnull(name)):
        first = name.split(' ')
        producerFirstNames.append(first[0])
        
print(len(set(producerFirstNames)))

1599
